In [1]:
import os 
import pandas as pd 

In [2]:
executions = range(1, 6) # amount of executions per raha configuration
# labeling_budget = [0.10, 0.25, 0.5, 0.75, 1, 2, 3, 5, 10, 15, 20]
n_cols = 1343 # amount of columns in the data lake
# labeling_budget_cells = [round(n_cols*x) for x in labeling_budget]
labeling_budget_cells = [158, 336, 672, 1008] # amount of labeling budgets Raha was run with (this row or the row above)
sandbox_path = "./raha/raha/datasets/DGov-141" # path towards the data lake (this dataset is not in this repository, but in another)
res_path = "./results/raha-dgov-141/exp_raha-non-enough-labels-dgov-141" # path towards the experiment folder, where the results are stored
df_path = "./raha/benchmark-results/DGov-141/raha-dgov-141-rt.csv" # path towards a csv file that contain benchmark results
tp_fn = 1082998 # amount of true positives and false negatives

In [3]:
import json

results = {"labeling_budget": [], "precision": [], "recall": [], "f_score": []}
for l in labeling_budget_cells:
    total_precision = 0
    total_recall = 0
    total_f_score = 0
    for exec in executions:
        tp = 0
        tp_fp = 0
        results_path_raha = os.path.join(res_path, "results_" + str(l) + "_"  + str(exec))
        for file in os.listdir(results_path_raha):
            if file.endswith(".json"):
                with open(os.path.join(results_path_raha, file)) as f:
                    json_content = json.load(f)
                    tp += json_content['tp']
                    tp_fp += json_content['ed_tpfp']
        precision = tp / tp_fp if tp_fp > 0 else 0
        recall = tp / tp_fn if tp_fn > 0 else 0
        f_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
        total_precision += precision
        total_recall += recall
        total_f_score += f_score
    avg_precision = total_precision / len(executions)
    avg_recall = total_recall / len(executions)
    avg_f_score = total_f_score / len(executions)
    results["labeling_budget"].append(l)
    results["precision"].append(avg_precision)
    results["recall"].append(avg_recall)
    results["f_score"].append(avg_f_score)

In [4]:
results_df = pd.DataFrame.from_dict(results)

In [5]:
results_df.to_csv(df_path, index=False)